In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load CSV using the second row (index 1) as the header
file_path = 'WaferPlan.csv' # Adjust path if necessary
try:
    # header=1 indicates the row at index 1 in the file is the header
    df = pd.read_csv(file_path, header=1)
    print(f"File '{file_path}' loaded successfully using row 1 as header.")
    # print(df.head(3)) # Uncomment to view the first few rows and verify columns
except Exception as e:
    print(f"Error loading '{file_path}': {e}")
    df = pd.DataFrame() # Create empty DataFrame if loading fails

File 'WaferPlan.csv' loaded successfully using row 1 as header.


In [3]:
# Remove fully empty columns (if any)
if not df.empty:
    df.dropna(axis=1, how='all', inplace=True)
    print(f"Shape after removing empty columns: {df.shape}")

Shape after removing empty columns: (4, 2)


In [4]:
df_filtered_waferplan = pd.DataFrame() # Initialize empty
if not df.empty:
    # Use the actual name of the first column found after loading with header=1
    id_col = 'Design Handle' # Adjusted based on previous analysis

    if id_col in df.columns:
        # Filter only by the Product IDs of interest in the first column
        condition = df[id_col].isin(['21A', '22B', '23C'])
        df_filtered_waferplan = df.loc[condition].copy()
        print(f"Rows filtered by '{id_col}'. Shape: {df_filtered_waferplan.shape}")
        # print(df_filtered_waferplan[[id_col]].head()) # Uncomment to view head
    else:
        print(f"Error: ID column '{id_col}' not found. Check file loading.")

Rows filtered by 'Design Handle'. Shape: (3, 2)


In [5]:
if not df_filtered_waferplan.empty:
    df_filtered_waferplan.drop_duplicates(inplace=True)

In [6]:
if not df_filtered_waferplan.empty:
    # Clean all column names, including the ID column and week columns
    df_filtered_waferplan.columns = df_filtered_waferplan.columns.str.strip().str.replace(" ", "_").str.replace(".", "_", regex=False).str.replace("-", "_")
    # print("Column names cleaned.") # Uncomment for confirmation
    # print(df_filtered_waferplan.columns[:15].tolist()) # Uncomment to view cleaned names

In [7]:
if not df_filtered_waferplan.empty:
    # Values start from the SECOND column (index 1)
    value_columns = df_filtered_waferplan.columns[1:]
    print(f"Attempting to convert {len(value_columns)} wafer plan columns to numeric...")
    for col in value_columns:
        df_filtered_waferplan[col] = pd.to_numeric(df_filtered_waferplan[col], errors='coerce')

    nan_count = df_filtered_waferplan[value_columns].isnull().sum().sum()
    if nan_count > 0:
        print(f"Warning: {nan_count} NaNs generated during numeric conversion of wafer plan.")
        # Consider filling NaNs if appropriate, e.g., with 0 for zero wafers planned
        # Example: df_filtered_waferplan[value_columns] = df_filtered_waferplan[value_columns].fillna(0)
    else:
        print("Numeric conversion completed without NaNs.")
    # print(df_filtered_waferplan.info()) # Uncomment to verify dtypes

Attempting to convert 1 wafer plan columns to numeric...


In [8]:
df_long_waferplan = pd.DataFrame() # Initialize empty
if not df_filtered_waferplan.empty:
    try:
        id_col_clean = df_filtered_waferplan.columns[0] # Cleaned ID col name ('Design_Handle')
        value_vars = df_filtered_waferplan.columns[1:] # Value columns start from index 1

        # Rename the ID column for the final output
        df_to_melt = df_filtered_waferplan.rename(columns={
            id_col_clean: 'Product_ID' # Final desired name for the ID column
        })

        # id_vars should only contain the renamed ID column
        df_long_waferplan = pd.melt(
            df_to_melt,
            id_vars=['Product_ID'],
            value_vars=value_vars,
            var_name='Week_Original',
            value_name='Wafer_Starts' # Name for the wafer start value
        )
        print(f"Wafer Plan DataFrame reshaped to long format. Final shape: {df_long_waferplan.shape}")
        # print(df_long_waferplan.head()) # Uncomment to view result
    except Exception as e:
      print(f"Error during Wafer Plan melt: {e}")

Wafer Plan DataFrame reshaped to long format. Final shape: (3, 3)


In [9]:
output_file = "WaferPlan_Long.csv"
if not df_long_waferplan.empty:
    df_long_waferplan.to_csv(output_file, index=False)
    print(f"Processed Wafer Plan data saved to '{output_file}'")
else:
    print("Final Wafer Plan DataFrame is empty, file not saved.")

Processed Wafer Plan data saved to 'WaferPlan_Long.csv'
